In [13]:
import pandas as pd
import numpy as np
import re

In [29]:
delit16 = pd.read_excel('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012(1).xlsx', sheet_name='Services PN 2016', header=2)

In [32]:
columns_paris = [x for x in delit16.columns.values if 'ARRONDISSEMENT' in x]
columns_paris

['DTSP75 01ER ARRONDISSEMENT',
 'DTSP75 02EME ARRONDISSEMENT',
 'DTSP75 03EME ARRONDISSEMENT',
 'DTSP75 04EME ARRONDISSEMENT',
 'DTSP75 05EME ET 06EME ARRONDISSEMENTS',
 'DTSP75 06EME ARRONDISSEMENT',
 'DTSP75 07EME ARRONDISSEMENT',
 'DTSP75 08EME ARRONDISSEMENT',
 'DTSP75 09EME ARRONDISSEMENT',
 'DTSP75 10EME ARRONDISSEMENT',
 'DTSP75 11EME ARRONDISSEMENT',
 'DTSP75 12EME ARRONDISSEMENT',
 'DTSP75 13EME ARRONDISSEMENT',
 'DTSP75 14EME ARRONDISSEMENT',
 'DTSP75 15EME ARRONDISSEMENT',
 'DTSP75 16EME ARRONDISSEMENT',
 'DTSP75 17EME ARRONDISSEMENT',
 'DTSP75 18EME ARRONDISSEMENT',
 'DTSP75 19EME ARRONDISSEMENT',
 'DTSP75 20EME ARRONDISSEMENT']

In [34]:
delit_paris_2016 = delit16.loc[:, columns_paris]

In [48]:
columns_929394 = [x for x in delit16.columns.values if (any(w in x for w in ['DTSP92', 'DTSP93', 'DTSP94']))]

In [49]:
delit_929394_2016 = delit16.loc[:, columns_929394]

In [65]:
delit_929394_2016.sum(axis=0).sort_values(ascending=False).head(50)

DTSP93 ST DENIS                  13714
DTSP93 AULNAY SOUS BOIS          10795
DTSP94 KREMLIN BICETRE           10222
DTSP93 AUBERVILLIERS              9286
DTSP94 CRETEIL                    8946
DTSP93 BOBIGNY                    8844
DTSP93 LES LILAS                  8622
DTSP92 BOULOGNE BILLANCOURT       8481
DTSP93 MONTREUIL                  8283
DTSP94 L'HAY LES ROSES            7358
DTSP93 LA COURNEUVE               7218
DTSP92 NANTERRE                   6782
DTSP92 LA DEFENSE                 6537
DTSP93 BONDY                      6018
DTSP93 PANTIN                     5800
DTSP92 LEVALLOIS PERRET           5725
DTSP93 STAINS                     5604
DTSP94 VITRY SUR SEINE            5514
DTSP92 ANTONY                     5415
DTSP93 VILLEPINTE                 5381
DTSP93 ST OUEN                    5307
DTSP92 NEUILLY SUR SEINE          5249
DTSP92 COLOMBES                   5228
DTSP93 DRANCY                     5103
DTSP93 EPINAY SUR SEINE           5098
DTSP92 ASNIERES SUR SEINE

In [86]:
names = delit16.iloc[:, 1].to_frame()
names.columns = ['name']

In [146]:
(pd.concat([pd.concat([names, delit_paris_2016], 1), delit_929394_2016], 1)
 .query('name != "Autres vols simples contre des particuliers dans des locaux ou lieux publics"')
).sum()[1:].sort_values(ascending=False).head(10)

DTSP75 18EME ARRONDISSEMENT              17988
DTSP93 ST DENIS                          13714
DTSP75 19EME ARRONDISSEMENT              13678
DTSP75 15EME ARRONDISSEMENT              13312
DTSP75 16EME ARRONDISSEMENT              11287
DTSP75 08EME ARRONDISSEMENT              11052
DTSP75 05EME ET 06EME ARRONDISSEMENTS    10867
DTSP75 17EME ARRONDISSEMENT              10535
DTSP75 20EME ARRONDISSEMENT              10155
DTSP93 AULNAY SOUS BOIS                  10135
dtype: object

In [88]:
eda1 = pd.concat([names, delit_paris_2016], axis=1)

In [103]:
eda2 = eda1.transpose()

In [106]:
eda2.columns = eda2.iloc[0, :]

In [111]:
eda2 = eda2.iloc[1:, :].apply(pd.to_numeric)

In [142]:
eda2.sum(numeric_only=True).sort_values(ascending=False)

name
Autres vols simples contre des particuliers dans des locaux ou lieux publics    61216
Vols à la tire                                                                  21693
Vols à la roulotte                                                              12379
Falsification et usages de chèques volés                                        10887
Escroqueries et abus de confiance                                               10677
                                                                                ...  
Index non utilisé                                                                   0
Index non utilisé                                                                   0
Marchandage - prêt de main d'oeuvre                                                 0
Homicides pour voler et à l'occasion de vols                                        0
Vols à main armée contre des entreprises de transports de fonds                     0
Length: 107, dtype: int64

In [151]:
from janitor import clean_names

In [171]:
pop = pd.read_csv('code-postal-code-insee-2015.csv', sep=';').clean_names()

In [174]:
pop.columns

Index(['geo_point', 'geo_shape', 'id_geofla', 'code_com', 'insee_com',
       'nom_com', 'statut', 'x_chf_lieu', 'y_chf_lieu', 'x_centroid',
       'y_centroid', 'z_moyen', 'superficie', 'population', 'code_cant',
       'code_arr', 'code_dept', 'nom_dept', 'code_reg', 'nom_reg', 'ligne_5',
       'libelle_acheminement', 'code_postal', 'nom_commune',
       'coordonnees_gps'],
      dtype='object')

In [180]:
pop = (pop
 .query('code_postal.astype("int").astype("str").str.contains("^93|^92|^94|^75")')
 .loc[:, ['code_postal', 'nom_com', 'population']]
)

,code_postal,nom_com,population
279,9400.0,SAURAT,668
297,9200.0,MONTJOIE-EN-COUSERANS,1053
322,9460.0,ROUZE,90
323,9270.0,MAZERES,3778
878,9240.0,DURBAN-SUR-ARIZE,178
...,...,...,...
39145,9230.0,TOURTOUSE,151
39157,9250.0,AXIAT,35
39204,7590.0,BORNE,38
39509,9310.0,LARNAT,21


In [182]:
pop.to_csv("insee-population2015.csv")

In [ ]:
pop.to_csv('insee')